In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint

/home/hao/anaconda3/envs/MLOntology/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#global variabls

directory_path = "/home/hao/AnacondaProjects/MLOntology/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"



In [3]:
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_july2017.txt"
read_label(label_file)

In [4]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptPairList.append([splitted[1], splitted[2].replace("\r\n", ""), 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = data_path + "ontHierarchy_july2017.txt"
read_pair(pair_file)

first2pairs = conceptPairList[10:15]
print(first2pairs)
print(len(conceptPairList))

[['37225000', '52860004', 1], ['159386001', '159385002', 1], ['233836002', '233835003', 1], ['233836002', '304914007', 1], ['224923003', '224717003', 1]]
502459


In [5]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                conceptNotPairList.append([splitted[0], splitted[1].replace("\r\n", ""), 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = data_path + "taxNotPairs_july2017.txt"
read_not_pair(notPair_file)

# first2pairs = {k: conceptNotPairDict[k] for k in list(conceptNotPairDict)[10:15]}
first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

# In-place shuffle
random.shuffle(conceptNotPairList)
conceptNotPairList = conceptNotPairList[:len(conceptPairList)]

print(len(conceptNotPairList))

[['273187009', '272765000', 0], ['272877001', '272765000', 0], ['273216002', '272765000', 0], ['273125004', '272765000', 0], ['272973003', '272765000', 0]]
6167243
502459


In [6]:
vector_model_file = vector_model_path + "model0"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[('237267007', 0.6096370220184326),
 ('722913002', 0.5996204018592834),
 ('722912007', 0.5989267230033875),
 ('267262008', 0.5890039205551147),
 ('277485007', 0.5662010312080383),
 ('446466006', 0.558641791343689),
 ('177131001', 0.5541282892227173),
 ('177130000', 0.5492905974388123),
 ('199672002', 0.5478367805480957),
 ('10759611000119105', 0.5416226983070374)]


In [7]:


def readFromPairList(id_pair_list, id_notPair_list):
    pair_list = id_pair_list + id_notPair_list
    random.shuffle(pair_list)
    idpairs_list =[]
    label_list =[]
    for i, line in enumerate(pair_list):      
        idpairs_list.append([line[0], line[1]])
        label_list.append(line[2])
    return idpairs_list, label_list

idpairs_list, label_list= readFromPairList(conceptPairList, conceptNotPairList)

print(label_list[:20])

[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0]


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(idpairs_list, label_list, test_size = 0.2, shuffle= True)
print(X_train[:20])
print(X_test[:20])
print(y_train[:20])
print(y_test[:20])

[['395150007', '418711005'], ['105241000', '7557006'], ['715705000', '8696009'], ['2649007', '61789006'], ['18307000', '272520006'], ['66742008', '438980004'], ['298350001', '24163000'], ['371430001', '417625002'], ['386033004', '276730002'], ['161043008', '88284004'], ['81876003', '67241009'], ['243872007', '170626004'], ['364567007', '364572003'], ['223697002', '223369002'], ['46743008', '74628008'], ['52807003', '107069004'], ['409109009', '353987005'], ['69527006', '104975003'], ['350345004', '347597002'], ['700774005', '465405006']]
[['1090009', '106234000'], ['253917003', '93406002'], ['90317004', '314468002'], ['241774007', '291954007'], ['405789005', '363905002'], ['715141007', '64770007'], ['468289004', '463072004'], ['698101006', '701219001'], ['87118001', '706912001'], ['53973008', '67017004'], ['371572003', '702683000'], ['26154000', '443778003'], ['402102001', '468288007'], ['462733000', '466303007'], ['41167007', '106412004'], ['237000000', '177128002'], ['167947004', '42

In [ ]:
# nb_classes = 2

# Convert class vectors to binary class matrices.


# Y_train = np.eye(nb_classes)[y_train]
# Y_test = np.eye(nb_classes)[y_test]

# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

In [105]:
feature_number = 1024

def getVectorFromModel(concept_id, conceptLabelDict, model):
    if concept_id in model.docvecs:
        concept_vector= model.docvecs[concept_id]
    else:
        print("%s not found, get inferred vector "%(concept_id))
        concept_label = conceptLabelDict[concept_id]
        concept_vector= model.infer_vector(concept_label.split())
    return concept_vector

def getVector(line, conceptLabelDict, model):        
    a = getVectorFromModel(line[0], conceptLabelDict, model)
    b = getVectorFromModel(line[1], conceptLabelDict, model)
    c = np.array((a, b))
    c = c.T 
    c = np.expand_dims(c, axis=2)
    print(c.shape)
    return c

In [106]:
nb_classes=2 

def generator(x_samples, y_samples, train_flag, batch_size=64):
    samples = list(zip(x_samples, y_samples))
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            X_samples = []
            Y_samples= []
            for batch_sample in batch_samples:
                pair_list = batch_sample[0]
                data_vector = getVector(pair_list, conceptLabelDict, vector_model)
                X_samples.append(data_vector)
                class_label = batch_sample[1] 
                Y_samples.append(class_label)
                
            X_samples = np.array(X_samples).astype('float32')
            Y_samples = np.eye(nb_classes)[Y_samples]
            print('one batch ready')
            yield shuffle(X_samples, Y_samples)
    

In [107]:
set_batch_size = 64

# compile and train the model using the generator function
train_generator = generator(X_train, y_train, train_flag=True, batch_size=set_batch_size)
validation_generator = generator(X_test, y_test, train_flag=False, batch_size=set_batch_size)

In [63]:
from keras import applications
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np

In [109]:
model = applications.resnet50.ResNet50(weights=None, include_top = True, classes=2)
print('Model loaded')

for layer in model.layers:
    layer.trainable = True
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Model loaded


In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18_cifar10.csv')

In [110]:
myepochs = 10

model.fit_generator(train_generator, 
                    steps_per_epoch=len(X_train)//set_batch_size, 
                    epochs=myepochs,
                    validation_data=validation_generator,
                    validation_steps=len(X_test)//set_batch_size,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

(512, 2, 1)
(512, 2, 1)Epoch 1/10

(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
one batch ready
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(512, 2, 1)
(

ValueError: Error when checking input: expected input_10 to have shape (224, 224, 3) but got array with shape (512, 2, 1)

In [ ]:
# model.fit(X_train, Y_train,
#               batch_size=batch_size,
#               nb_epoch=nb_epoch,
#               validation_data=(X_test, Y_test),
#               shuffle=True,
#               callbacks=[lr_reducer, early_stopper, csv_logger])

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = '/home/hao/Pictures/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [130]:


a = np.zeros((512,2))
print(a)
print(a.shape)
b = a.reshape(4,4,-1)
print(b)
print(b.shape)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
(512, 2)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
(4, 4, 64)
